# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up graphlab create

Make sure you have the latest version of graphlab (>= 1.7)

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to cbisho12@utk.edu and will expire on September 08, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1476399498.log


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create thepredictions vector by using np.dot()
    predictions =  np.dot(feature_matrix, weights)
    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


To normalize, apply element-wise division:

In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix/norms
    return(normalized_features, norms)

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction =  predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [23]:
k = len(weights)
ro = np.zeros(k)
for i in xrange(k):
    ro[i] = (simple_feature_matrix[:,i]*(output - prediction + simple_feature_matrix[:,i]*weights[i])).sum()
print ro    

[ 79400300.03492916  87939470.77299108  80966698.67596565]


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [24]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:,i]*(output - prediction + feature_matrix[:,i]*weights[i])).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [25]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [44]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    converged = False 
    weights = np.array(initial_weights) # make sure it's a numpy array
    while not converged:
        large_step=0.0
        for i in range(len(weights)):
            old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            step_size = abs(old_weights_i - weights[i])
            if large_step < step_size:
                large_step = step_size
                print step_size
        if large_step < tolerance:
            converged = True
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [40]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`

In [33]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [45]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

79400304.6581
9138168.3665
8194809.40937
6598905.03827
5522173.1943
4621129.81031
3867108.10625
3236118.81061
2708086.93955
2266213.1094
1896439.06265
1587000.40319
1328052.31095
1111356.328
930018.251241
778268.79269
651280.029038
545011.800818
456083.174344
381664.876995
319389.283634
267275.090398
223664.279321
187169.368346
156629.268444
131072.343461
109685.497423
91788.3058116
76811.3677895
64278.190664
53790.0302253
45013.2046617
37668.4784415
31522.1784089
26378.7594496
22074.5832053
18472.7119036
15458.5516701
12936.2067131
10825.4283904
9059.062091
7580.91070477
6343.94670622
5308.81596931
4442.58571219
3717.69673771
3111.08663488
2603.45604623
2178.65465672
1823.16737024
1525.68432523
1276.74107067
1068.41745343
894.085638065
748.199241441
626.116874095
523.954474047
438.461734928
366.918697163
307.049212273
256.948526997
215.022683285
179.937806498
150.577667948
126.008172072
105.447638057
88.2419305891
73.8436484337
61.7947089002
51.7117738649
43.2740537636
36.2131017707
3

In [46]:
prediction = predict_output(normalized_simple_feature_matrix, weights)
ris = prediction  - output
RSS = (ris*ris).sum()
print  RSS

1.63049248148e+15


In [47]:
print weights

[ 21624998.36636292  63157246.78545421         0.        ]


***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset?
2. Which features had weight zero at convergence?

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [48]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [49]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [52]:
my_output = 'price'
(all_feature_matrix, output) = get_numpy_data(train_data, all_features, my_output)
(normalized_all_feature_matrix, all_norms) = normalize_features(all_feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [56]:
sz = len(all_features)
initial_weights = np.zeros(sz+1)
l1_penalty = 1e7
tolerance = 1.0

In [57]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

71114625.7528
5024356.36869
5012324.91395
5000676.42086
4991243.17314
4984361.34335
4979504.92921
4973806.56525
2629220.52555
2879222.99938
1678495.17891
2706150.76622
1431139.48146
2532295.64864
1130424.02235
1337213.82745
2368316.57782
876486.765932
1421557.25966
2213124.35067
658897.06496
947126.043731
1552443.60013
507838.033159
438814.370591
371479.622608
313415.105581
264244.832139
222753.8608
187770.667306
158280.093357
133420.901606
112465.988296
94802.2134519
79912.6874505
67361.6926196
56781.942492
47863.835752
40346.3966195
34009.6378518
28668.1231465
24165.5406129
20370.1285268
17170.8195088
14473.990295
12200.7219837
10284.4905855
8669.22029242
7307.64249856
6159.91255101
5192.44375235
4376.92449333
3689.48975358
3110.02272544
2621.56612397
2209.82595597
1862.75322643
1570.19134162
1323.57889082
1115.69910862
940.46868654
792.759753529
668.249816086
563.295266587
474.824758224
400.249326423
337.386626367
284.397069883
239.729992345
202.078274757
170.340092748
143.586673193

***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [58]:
l1_penalty = 1e8
weights1e8 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

71114625.7528


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [59]:
l1_penalty = 1e4
tolerance=5e5
weights1e4 = lasso_cyclical_coordinate_descent(normalized_all_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

71114625.7528
656582.579069
2468988.04934
3584457.8054
4916244.61106
4931522.9822
5297036.06628
8464015.94738
54673.1764821
1604397.23552
3177762.60019
3587867.56036
4184746.78103
6818451.35869
11879.4066318
964887.595026
2803628.26512
3251015.14864
5570165.21901
35715.0766581
461708.794851
2459608.52475
2481794.0861
4559838.46759
48341.7961846
62854.599619
2149118.85088
3732056.69715
57620.7032791
251303.971005
1870640.79256
3053211.61353
65319.7893941
496015.27904
1621553.26546
1632779.1608
2497517.43558
71864.8638116
683815.925687
1399150.69309
1511868.85441
2043868.97325
77429.5462091
825115.958732
1200858.26714
1419417.42925
1674717.70906
82137.068367
928563.585831
1024278.85145
1349317.31443
1375420.75692
86093.5053087
1001332.75431
1296621.8143
89393.238424
1049362.65118
1257340.9361
92120.3563358
1077559.2824
1228271.24819
94349.5503441
1089965.38589
1206854.36471
96146.9540437
1089903.57062
1191059.40598
97570.9553257
1070096.76332
1179030.77017
98640.5203591
1052991.86154
116

In [61]:
print weights1e7
print weights1e8
print weights1e4
print all_features

[ 24429600.60933314         0.                 0.          48389174.35227978
         0.                 0.           3317511.16271982   7329961.9848964
         0.                 0.                 0.                 0.
         0.                 0.        ]
[ 71114625.75280938         0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.                 0.                 0.
         0.                 0.        ]
[ 77779073.91265215 -22884012.25023361  15348487.08089997
  92166869.69883084  -2139328.0824278   -8818455.54409496
   6494209.73310655   7065162.05053197   4119079.21006765
  18436483.52618778 -14566678.54514349  -5528348.75179429
 -83591746.20730527   2784276.46012858]
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [62]:
(features, norms) = normalize_features(all_feature_matrix) # normalize features

In [67]:
l1_penalty = 1e7
tolerance=1.
weights1e7_normalized = lasso_cyclical_coordinate_descent(features, output,
                                            initial_weights, l1_penalty, tolerance)
weights1e7_normalized = weights1e7_normalized/norms

71114625.7528
5024356.36869
5012324.91395
5000676.42086
4991243.17314
4984361.34335
4979504.92921
4973806.56525
2629220.52555
2879222.99938
1678495.17891
2706150.76622
1431139.48146
2532295.64864
1130424.02235
1337213.82745
2368316.57782
876486.765932
1421557.25966
2213124.35067
658897.06496
947126.043731
1552443.60013
507838.033159
438814.370591
371479.622608
313415.105581
264244.832139
222753.8608
187770.667306
158280.093357
133420.901606
112465.988296
94802.2134519
79912.6874505
67361.6926196
56781.942492
47863.835752
40346.3966195
34009.6378518
28668.1231465
24165.5406129
20370.1285268
17170.8195088
14473.990295
12200.7219837
10284.4905855
8669.22029242
7307.64249856
6159.91255101
5192.44375235
4376.92449333
3689.48975358
3110.02272544
2621.56612397
2209.82595597
1862.75322643
1570.19134162
1323.57889082
1115.69910862
940.46868654
792.759753529
668.249816086
563.295266587
474.824758224
400.249326423
337.386626367
284.397069883
239.729992345
202.078274757
170.340092748
143.586673193

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [68]:
print weights1e7_normalized[3]

161.317456248


In [69]:
l1_penalty = 1e8
tolerance=1.
weights1e8_normalized = lasso_cyclical_coordinate_descent(features, output,
                                            initial_weights, l1_penalty, tolerance)
weights1e8_normalized = weights1e8_normalized/norms

71114625.7528


In [70]:
l1_penalty = 1e4
tolerance=5e5
weights1e4_normalized = lasso_cyclical_coordinate_descent(features, output,
                                            initial_weights, l1_penalty, tolerance)
weights1e4_normalized = weights1e4_normalized/norms

71114625.7528
656582.579069
2468988.04934
3584457.8054
4916244.61106
4931522.9822
5297036.06628
8464015.94738
54673.1764821
1604397.23552
3177762.60019
3587867.56036
4184746.78103
6818451.35869
11879.4066318
964887.595026
2803628.26512
3251015.14864
5570165.21901
35715.0766581
461708.794851
2459608.52475
2481794.0861
4559838.46759
48341.7961846
62854.599619
2149118.85088
3732056.69715
57620.7032791
251303.971005
1870640.79256
3053211.61353
65319.7893941
496015.27904
1621553.26546
1632779.1608
2497517.43558
71864.8638116
683815.925687
1399150.69309
1511868.85441
2043868.97325
77429.5462091
825115.958732
1200858.26714
1419417.42925
1674717.70906
82137.068367
928563.585831
1024278.85145
1349317.31443
1375420.75692
86093.5053087
1001332.75431
1296621.8143
89393.238424
1049362.65118
1257340.9361
92120.3563358
1077559.2824
1228271.24819
94349.5503441
1089965.38589
1206854.36471
96146.9540437
1089903.57062
1191059.40598
97570.9553257
1070096.76332
1179030.77017
98640.5203591
1052991.86154
116

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [65]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [71]:
prediction = predict_output(test_feature_matrix, weights1e4_normalized)
ris = prediction  - test_output
RSS = (ris*ris).sum()
print  RSS

2.2778100476e+14


In [72]:
prediction = predict_output(test_feature_matrix, weights1e8_normalized)
ris = prediction  - test_output
RSS = (ris*ris).sum()
print  RSS

5.37166150034e+14


In [73]:
prediction = predict_output(test_feature_matrix, weights1e7_normalized)
ris = prediction  - test_output
RSS = (ris*ris).sum()
print  RSS

2.75962079909e+14


***QUIZ QUESTION***

Which model performed best on the test data?